<a href="https://colab.research.google.com/github/brunoartu07-creator/dados-/blob/main/regressao_agro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

regressão dados em painel

In [1]:
# instalar a biblioteca (Painel Data Models)
!pip install linearmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 8.0 MB/s eta 0:00:00


In [28]:
# estimacao estatistica efeitos fixos

import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS
import statsmodels.api as sm


# 1. Carregar base de dados

df = pd.read_excel('matriz de dados nordeste.xlsx')

# Limpar e Renomear colunas
df.columns = df.columns.str.strip() # Remove extra spaces

# DEBUG: Imprima as colunas antes de renomear para verificar o nome exatos
print("Columns after stripping whitespace:", df.columns)
# ------------------------------------------------------------------

df = df.rename(columns={
    'ESTADOS': 'estado', # Corrected: 'ESTADOS' to 'estado'
    'ANO': 'ano', # Corrected: 'ANO' (uppercase) to 'ano' (lowercase)
    'LN_INVESTIMENTO_RURAL': 'ln_investimento',
    'LN_CREDITO_RURAL': 'ln_credito',
    'LN_BIB_AGRICOLA': 'ln_pib',
    'LN_INCLUSAO_FINANCEIRA_QT_CREDITO RURAL': 'ln_inclusao', # Corrected column name
    'LN_INFRAESTRUTURA_ARMAZEM_TONELADA': 'ln_infraestrutura'
})

# 2. Definir estrutura de painel

df = df.set_index(['estado', 'ano'])

# 3. Transformar em log (colunas já estão em log, apenas ajustando nomes)

# As colunas já vêm log-transformadas e foram renomeadas acima.

# ===============================
# 4. Definir variáveis
# ===============================
Y = df['ln_investimento']

X = df[['ln_credito',
        'ln_pib',
        'ln_inclusao',
        'ln_infraestrutura']]

X = sm.add_constant(X)


# 5. Rodar modelo com efeitos fixos

modelo_fe = PanelOLS(
    Y,
    X,
    entity_effects=True,  # efeitos fixos por estado
    time_effects=True     # efeitos fixos por ano
)

resultado = modelo_fe.fit(cov_type='clustered', cluster_entity=True)


# 6. Exibir resultado

print(resultado.summary)


Columns after stripping whitespace: Index(['ESTADOS', 'ANO', 'LN_INVESTIMENTO_RURAL', 'LN_CREDITO_RURAL',
       'LN_BIB_AGRICOLA', 'LN_INCLUSAO_FINANCEIRA_QT_CREDITO RURAL',
       'LN_INFRAESTRUTURA_ARMAZEM_TONELADA'],
      dtype='object')
                          PanelOLS Estimation Summary                           
Dep. Variable:        ln_investimento   R-squared:                        0.5628
Estimator:                   PanelOLS   R-squared (Between):              0.9632
No. Observations:                 108   R-squared (Within):               0.6459
Date:                Tue, Feb 17 2026   R-squared (Overall):              0.9257
Time:                        22:05:17   Log-likelihood                    114.39
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      27.029
Entities:                           9   P-value                           0.0000
Avg Obs:                    

In [17]:
tabela = pd.DataFrame({
    'Coeficiente': resultado.params,
    'Erro-padrão': resultado.std_errors,
    'Estatística t': resultado.tstats,
    'p-valor': resultado.pvalues
})

print(tabela)


                   Coeficiente  Erro-padrão  Estatística t   p-valor
const                 3.080288     1.721999       1.788786  0.077254
ln_credito            0.381751     0.082016       4.654584  0.000012
ln_pib                0.106795     0.065496       1.630570  0.106725
ln_inclusao           0.627668     0.136472       4.599255  0.000015
ln_infraestrutura     0.000313     0.002956       0.105915  0.915903


In [5]:
# tabela para o excel
tabela.to_excel('tabela_resultados_FE.xlsx')


In [18]:
# modelo de hausman
from linearmodels.panel import RandomEffects

# Modelo de efeitos aleatórios
modelo_re = RandomEffects(Y, X)
resultado_re = modelo_re.fit()

print(resultado_re.summary)


                        RandomEffects Estimation Summary                        
Dep. Variable:        ln_investimento   R-squared:                        0.8559
Estimator:              RandomEffects   R-squared (Between):              0.9652
No. Observations:                 108   R-squared (Within):               0.7697
Date:                Tue, Feb 17 2026   R-squared (Overall):              0.9421
Time:                        21:41:23   Log-likelihood                    58.977
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      152.89
Entities:                           9   P-value                           0.0000
Avg Obs:                       12.000   Distribution:                   F(4,103)
Min Obs:                       12.000                                           
Max Obs:                       12.000   F-statistic (robust):             152.89
                            

In [19]:
# continuação do teste de hausman comparacao FE and RE
from linearmodels.panel import compare

print(compare({'FE': resultado, 'RE': resultado_re}))


                        Model Comparison                       
                                         FE                  RE
---------------------------------------------------------------
Dep. Variable               ln_investimento     ln_investimento
Estimator                          PanelOLS       RandomEffects
No. Observations                        108                 108
Cov. Est.                         Clustered          Unadjusted
R-squared                            0.5628              0.8559
R-Squared (Within)                   0.6459              0.7697
R-Squared (Between)                  0.9632              0.9652
R-Squared (Overall)                  0.9257              0.9421
F-statistic                          27.029              152.89
P-value (F-stat)                     0.0000              0.0000
=====================     =================   =================
const                                3.0803             -0.0396
                                   (1.78

In [20]:
# teste de hausman forma direta apenas P-VALOR and estatistica Hausman
import numpy as np
import pandas as pd
from scipy import stats


b_fe = resultado.params
b_re = resultado_re.params

# Covariâncias
cov_fe = resultado.cov
cov_re = resultado_re.cov

# Diferença dos coeficientes
b_diff = b_fe - b_re

# Diferença das covariâncias
cov_diff = cov_fe - cov_re

# Estatística de Hausman
stat = np.dot(np.dot(b_diff.T, np.linalg.inv(cov_diff)), b_diff)

# Graus de liberdade
df = len(b_diff)

# p-valor
p_value = 1 - stats.chi2.cdf(stat, df)

print("Estatística Hausman:", stat)
print("p-valor:", p_value)


Estatística Hausman: 0.016902381831601088
p-valor: 0.9999980361983066


In [21]:
# teste de wooldridge
import pandas as pd
import statsmodels.api as sm
from scipy import stats

# Wooldridge teste de autocorrelacao.
# H0: nenhuma correlacao serial de primeira ordem.

residuals = resultado.resids.to_frame(name='residuals')


residuals = residuals.sort_index()

# Atrasar os resíduos dentro de cada entidade (estado)
residuals['lag_residuals'] = residuals.groupby(level='estado')['residuals'].shift(1)

# Eliminar valores NaN criados por atraso

df_wooldridge = residuals.dropna()

#  Define Y_aux e X_aux para a regressão auxiliar.

Y_aux = df_wooldridge['residuals']
X_aux = df_wooldridge['lag_residuals']
X_aux = sm.add_constant(X_aux) # Add a constant to the auxiliary regression

#  Executa uma regressão auxiliar usando OLS (Mínimos Quadrados Ordinários) (pooled)

aux_model = sm.OLS(Y_aux, X_aux)
aux_results = aux_model.fit()

# Calculo da estatística de teste de Wooldridge
# A estatística de teste é N * R^2 da regressão auxiliar.
# onde N é o número de observações na regressão auxiliar.
N_aux = aux_results.nobs
R_squared_aux = aux_results.rsquared

# teste de wooldridge
wooldridge_stat = N_aux * R_squared_aux

# p-value (Os graus de liberdade são normalmente iguais ao número de resíduos defasados, que neste caso é 1.)
p_value_wooldridge = 1 - stats.chi2.cdf(wooldridge_stat, 1)

print("Wooldridge Test for Autocorrelation:")
print(f"Test Statistic: {wooldridge_stat:.4f}")
print(f"P-value: {p_value_wooldridge:.4f}")

if p_value_wooldridge < 0.05:
    print("Conclusion: Reject the null hypothesis. There is evidence of first-order serial correlation.")
else:
    print("Conclusion: Fail to reject the null hypothesis. No evidence of first-order serial correlation.")

Wooldridge Test for Autocorrelation:
Test Statistic: 9.5287
P-value: 0.0020
Conclusion: Reject the null hypothesis. There is evidence of first-order serial correlation.


In [22]:
# teste de breusch-pagan
import statsmodels.stats.api as sms

# Converter objetos PanelData em arrays NumPy
# Para resíduos
residuals_np = resultado.resids.values
# Para exog, acesse o dataframe subjacente e, em seguida, seus valores.
exog_np = resultado.model.exog.dataframe.values

bp_test = sms.het_breuschpagan(residuals_np, exog_np)
print(bp_test)

(np.float64(2.913397429432099), np.float64(0.5724211673832216), np.float64(0.7138872317953107), np.float64(0.5842899595804166))


In [27]:
# teste de pesaran (implementação manual devido a ModuleNotFoundError)
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS
import statsmodels.api as sm


# 1. Carregar base de dados (added for self-containment)

df = pd.read_excel('matriz de dados nordeste.xlsx')

# Limpar e Renomear colunas (added for self-containment)
df.columns = df.columns.str.strip()

df = df.rename(columns={
    'ESTADOS': 'estado',
    'ANO': 'ano',
    'LN_INVESTIMENTO_RURAL': 'ln_investimento',
    'LN_CREDITO_RURAL': 'ln_credito',
    'LN_BIB_AGRICOLA': 'ln_pib',
    'LN_INCLUSAO_FINANCEIRA_QT_CREDITO RURAL': 'ln_inclusao',
    'LN_INFRAESTRUTURA_ARMAZEM_TONELADA': 'ln_infraestrutura'
})


# 2. Definir estrutura de painel (added for self-containment)

df = df.set_index(['estado', 'ano'])

# Redefinindo o resultado aqui para garantir que esteja disponível para o teste.
Y = df['ln_investimento']

X = df[['ln_credito',
        'ln_pib',
        'ln_inclusao',
        'ln_infraestrutura']]

X = sm.add_constant(X)

modelo_fe = PanelOLS(
    Y,
    X,
    entity_effects=True,
    time_effects=True
)

resultado = modelo_fe.fit(cov_type='clustered', cluster_entity=True)

def pesaran_cd_manual(residuals_panel):
    """
    Calculates the Pesaran CD test for cross-sectional dependence.

    Args:
        residuals_panel (pd.Series): Panel residuals (MultiIndex: entity, time).

    Returns:
        tuple: CD test statistic and p-value.
    """
    # Certifique-se de que os resíduos estejam classificados por entidade e, em seguida, por tempo.
    residuals_panel = residuals_panel.sort_index()

    entities = residuals_panel.index.get_level_values(0).unique()
    T = residuals_panel.groupby(level=0).size().iloc[0] # Assuming balanced panel for simplicity

    N = len(entities)
    sum_rho_ij = 0
    num_pairs = 0

    # Calcular coeficientes de correlação par a par.
    for i in range(N):
        for j in range(i + 1, N):
            entity_i = entities[i]
            entity_j = entities[j]

            res_i = residuals_panel.loc[entity_i]
            res_j = residuals_panel.loc[entity_j]

            # Calcular a correlação
            if len(res_i) > 1 and len(res_j) > 1:
                rho_ij = res_i.corr(res_j)
                if not pd.isna(rho_ij):
                    sum_rho_ij += rho_ij
                    num_pairs += 1

    if num_pairs == 0:
        return np.nan, np.nan # No pairs to calculate CD test

    # Pesaran CD teste estatistica
    cd_stat = np.sqrt(2 * T / (N * (N - 1))) * sum_rho_ij

    # p-value using standard normal distribution
    from scipy.stats import norm
    p_value = 2 * (1 - norm.cdf(np.abs(cd_stat)))

    return cd_stat, p_value

# Get residuals from the FE model
residuals = resultado.resids

# execute o manual teste Pesaran CD
cd_stat, p_value_cd = pesaran_cd_manual(residuals)

print("Pesaran CD Test for Cross-sectional Dependence:")
print(f"CD Test Statistic: {cd_stat:.4f}")
print(f"P-value: {p_value_cd:.4f}")

if p_value_cd < 0.05:
    print("Conclusion: Reject the null hypothesis. There is evidence of cross-sectional dependence.")
else:
    print("Conclusion: Fail to reject the null hypothesis. No evidence of cross-sectional dependence.")

Pesaran CD Test for Cross-sectional Dependence:
CD Test Statistic: -2.5066
P-value: 0.0122
Conclusion: Reject the null hypothesis. There is evidence of cross-sectional dependence.


In [24]:
# esse codigo usa se houver autocorrelação e heterocedasticidade
cov_type='clustered'
cluster_entity=True
